In [1]:
#Import Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import numpy as np
%matplotlib inline

# Custom Function and Transformers

In [2]:
#Function to Import Data

def import_data(path, drop, sep,dropna=True, drop_duplicates=True):
    data = pd.read_csv(path, sep)
    print("Banyaknya baris dan kolom", data.shape)
    print("Banyaknya data duplicate", data.duplicated().sum())
    data = data.drop_duplicates() if drop_duplicates else data
    data = data.dropna() if dropna else data
    data = data.drop(drop,axis=1)
    print("Banyaknya data setelah di drop", data.shape)
    #print(data.head())
    return data

In [3]:
#Function to Extract Input and Output

def extract_input_output(data, output):
    y = data[output]
    x = data.drop(output, axis=1)
    #print(x.columns) #optional
    #print(y.head()) #optional
    return x,y

In [4]:
#function to split train test data (default test_size = 0.4)
from sklearn.model_selection import train_test_split

def train_test_data(x,y, _test_size = 0.4): 
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =_test_size,
                                                        random_state =100)
   # print(x_train.head(), y_train.head()) #
    #print(x_test.head(), y_test.head()) #
    return x_train, x_test, y_train, y_test

In [5]:
#Function to evaluate score
from sklearn.metrics import accuracy_score

def scoring(model) :
    y_predict = model.predict(x_train)
    print("Model Score based on training data")
    print(accuracy_score(y_train, y_predict))
    y_predict = model.predict(x_test)
    print("Model Score based on test data")
    print(accuracy_score(y_test, y_predict))


In [6]:
from sklearn.base import TransformerMixin #gives fit_transform method for free
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X[self.attribute_names].values

# Visualization Functions

In [8]:
#Make sure y_train is balanced
def plot_y_balance(y, df) : 
  ax = sns.countplot(y,label="Count")       # M = 212, B = 357
  C, N = y.value_counts()
  print('Number of Churned: ',C)
  print('Number of Not Churned : ',N) 
  df.isChurned.value_counts(normalize=True)

# Pre Processing Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import FeatureUnion

num_attribs = list(x_train.drop('userLevel',axis=1))
cat_attribs = ["userLevel"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    #('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
  ])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', MyLabelBinarizer()),
  ])

preprocessor = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
  ])

# Machine Learning Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('processor', preprocessor), #Step1 - preprocess data
    ('clf', LogisticRegression()) #step2 - classifier (default : LogisticRegression )
])

from sklearn.model_selection import cross_validate

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = []
clfs.append(LogisticRegression())
# clfs.append(SVC())
# clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
# clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, x_train, y_train,cv=3)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())